# CloudEx Algo Main Script

#### Default Imports

In [1]:
import sys 
sys.path

new_paths = ['/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/root/CloudExchange', '/root/']
for p in new_paths : 
    sys.path.append(p) 


In [2]:
sys.path

['/root/cs349f/implementation1',
 '/root/.pyenv/versions/3.7.7/lib/python37.zip',
 '/root/.pyenv/versions/3.7.7/lib/python3.7',
 '/root/.pyenv/versions/3.7.7/lib/python3.7/lib-dynload',
 '',
 '/home/steam1994/.local/lib/python3.7/site-packages',
 '/root/.pyenv/versions/3.7.7/lib/python3.7/site-packages',
 '/home/steam1994/.local/lib/python3.7/site-packages/IPython/extensions',
 '/home/steam1994/.ipython',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/root/CloudExchange',
 '/root/']

In [20]:
# Import packages.
import datetime
import json
import os
import sys
import time

import pandas as pd
import numpy as np
import redis
from pandas.core.common import SettingWithCopyWarning

import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# CloudEx imports.
import cloud_ex

# Import AlgorithmicTrader helper class.
from algorithmic_trader_shay import AlgorithmicTrader
from algorithmic_trader_shay import summarize_historical_trades_df

# Start Redis and its Python API.
os.system("redis-server --daemonize yes")
time.sleep(1)

# Get CloudEX and VM-specific config. 
# NOTE: gateway_ip will be null when the exchange is not online 
def get_vm_config():
    with open("/root/vm_config.json", "r") as read_file:
        config = json.load(read_file)
    return config

config = get_vm_config()


# utilities  

ORDER_FIELDS_LIST = [
    'Symbol', 'OrderID', 'CancelID', 'ClientID', 'OrderType', 'OrderAction',
    'SubmitTimestamp', 'GatewayTimestamp', 'EnqueueTimestamp',
    'DequeueTimestamp', 'OrderSerialNum', 'LimitPrice', 'ResultType','NumShares'
]

TRADE_FIELDS_LIST = [
    "Symbol", "BuyerSerialNum", "SellerSerialNum", "BuyerOrderID",
    "SellerOrderID", "BuyerClientID", "SellerClientID", "ExecPrice",
    "CashTraded", "SharesTraded", "CreationTimestamp", "ReleaseTimestamp",
    "TradeSerialNum"
]

'''
Takes in a cloud_ex.VectorOrder with serialized orders and returns a DataFrame
'''
def OrderDF(order_vec):
    if not len(order_vec):
        return pd.DataFrame(columns=ORDER_FIELDS_LIST)
    df = pd.DataFrame(order_vec).applymap(lambda x:x.SerializeOrder())[0].str.split('|', expand=True)
    df.columns = ORDER_FIELDS_LIST
    for label in ['SubmitTimestamp', 'GatewayTimestamp', 'EnqueueTimestamp',
                  'DequeueTimestamp', 'OrderSerialNum', 'LimitPrice','NumShares']:
        df.loc[:, label] = pd.to_numeric(df[label], errors='coerce')
    return df

'''
Takes in a cloud_ex.VectorOrder with serialized trades and returns a DataFrame
'''
def TradeDF(trade_vec):
    if not len(trade_vec):
        return pd.DataFrame(columns=TRADE_FIELDS_LIST)
    df = pd.DataFrame(trade_vec).applymap(lambda x:x.SerializeTrade())[0].str.split('|', expand=True)
    df.columns = TRADE_FIELDS_LIST
    for label in ["ExecPrice", "CashTraded", "SharesTraded",
                  "CreationTimestamp", "ReleaseTimestamp", "TradeSerialNum"]:
        df.loc[:, label] = pd.to_numeric(df[label], errors='coerce')
    return df

'''
Takes in a cloud_ex.MapStringOrder mapping Order ID strings to outstanding the coorresponding orders, 
and returns a DataFrame
'''
def OutstandingOrderDF(outstanding_orders):
    if not len(outstanding_orders):
        return pd.DataFrame(columns=ORDER_FIELDS_LIST)
    df = (pd.DataFrame(outstanding_orders.items())[1]).apply(lambda x:x.SerializeOrder()).str.split('|', expand=True)
    df.columns = ORDER_FIELDS_LIST
    for label in ['SubmitTimestamp', 'GatewayTimestamp', 'EnqueueTimestamp',
                  'DequeueTimestamp', 'OrderSerialNum', 'LimitPrice','NumShares']:
        df.loc[:, label] = pd.to_numeric(df[label], errors='coerce')
    return df 

In [4]:
config

{'gateway_ip': None,
 'client_id': 'C15',
 'client_token': 'dyjpqwzyvminsvjpxtyasxdnzpnqkuux',
 'project_id': 'jinkun-pro1',
 'bigtable_id': 'test-bt',
 'table_name': 'record-tb'}

#### Custom Imports

In [5]:
import importlib 
import sys 
#importlib.reload(sys.modules['mean_reversion_shay'])

In [6]:
import strategies_shay   
import threading
import utilities as u 
default_symbols = ['AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ', 'AK', 'AL', 'AM', 'AN', 'AO', 'AP', 'AQ', 'AR', 'AS', 'AT', 'AU', 'AV', 'AW', 'AX', 'AY', 'AZ', 'BA', 'BB', 'BC', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BK', 'BL', 'BM', 'BN', 'BO', 'BP', 'BQ', 'BR', 'BS', 'BT', 'BU', 'BV', 'BW', 'BX', 'BY', 'BZ', 'CA', 'CB', 'CC', 'CD', 'CE', 'CF', 'CG', 'CH', 'CI', 'CJ', 'CK', 'CL', 'CM', 'CN', 'CO', 'CP', 'CQ', 'CR', 'CS', 'CT', 'CU', 'CV', 'CW', 'CX', 'CY', 'CZ', 'DA', 'DB', 'DC', 'DD', 'DE', 'DF', 'DG', 'DH', 'DI', 'DJ', 'DK', 'DL', 'DM', 'DN', 'DO', 'DP', 'DQ', 'DR', 'DS', 'DT', 'DU', 'DV']
default_top_symbols = ['CQ', 'BV', 'BP', 'BY', 'CR', 'BL', 'BO', 'CX', 'DE', 'BF']

In [7]:
trader = None

#### Get the trader object

In [8]:
def getTrader() : 
    global trader
    if trader : 
        u.debug("Returning saved trader")
        return trader 

    # Get relevant fields from VM-specific config. Token is yours only, so don't make it public.
    gateway_ip = config["gateway_ip"]
    client_id = config["client_id"]
    client_token = config["client_token"]

    # Clear any existing data locally.
    redis_api = redis.Redis()
    redis_api.flushall();

    # Create CloudEx base trader object.
    trader = cloud_ex.Trader(gateway_ip, client_id, client_token)
    return trader 

def getSymbols() : 
    trader = getTrader() 
    return trader.GetSymbols()


def getPortfolio(): 
    portfolio_mat = cloud_ex.MapStringInt()
    return trader.GetPortfolioMatrix(portfolio_mat)
    

In [ ]:
T = getTrader()

In [ ]:
config['gateway_ip']

#### Main Todos



#### TRADING PARAMETERS

In [9]:
GLOBALS = { 
    'NUM_SHARES' :  1 , 
    'BIN_INTERVAL_MS' : 500,  #interval to bin the data with 
    'WAIT_INTERVAL_MS' : 500, 
    'BACKTEST_LOOKBACK_PERIOD_SECONDS' : 60*5  , #amount of historical data to backtest on 
    'MAX_NUM_ORDERS' : 60*5*2 , #how long the algo will trade for, in # of bins 
} 

### Selecting symbols to trade on based on aggregate volume over last n seconds  

We need to figure out which symbols to trade on prior to the initiation of trading. It seems natural to rank them by some metric and then take the top N of them, for example volume 

In [10]:
def aggregate_volume(symbol, seconds_in_past) : 
    """
    Gets the most recent 'seconds_in_past' historical data for the symbol and compute the total 
    'CashTraded' 
    """
    u.debug("Getting aggregate volume for {}".format(symbol))
    end_time_ms = int(time.time()*1e3)
    start_time_ms = end_time_ms - int(seconds_in_past*1e3)
    symbol_trades_vec = cloud_ex.VectorTrade()  
    cloud_ex.MarketDataAPI.PullTrades(config['project_id'], config['bigtable_id'], 
                                          config['table_name'], symbol, start_time_ms, 
                                          end_time_ms, symbol_trades_vec)
    sym_df = TradeDF(symbol_trades_vec)
    return np.sum(sym_df['CashTraded'])    

In [11]:
def rank_symbols_by_volume(symbols , seconds_in_past) : 
    data = [ [symbol, aggregate_volume(symbol,seconds_in_past)  ] for  symbol in symbols ] 
    data.sort(key=lambda x:  x[1] , reverse=True )
    return data

In [12]:
def get_top_n_symbols_by_volume(symbols,seconds_in_past, n )  : 
    ranked = rank_symbols_by_volume(symbols,seconds_in_past) 
    
    syms = [ x[0] for x in ranked[0:n]] 
    return (syms , ranked)  

In [ ]:
top_symbols, symbol_ranks = get_top_n_symbols_by_volume(default_symbols, 60*5, 10)

In [ ]:
top_symbols

In [ ]:
symbol_ranks

### QUERY ORDER HISTORY

In [13]:
### Order Monitoring 
def outstanding_orders() : 
    outstanding_orders = cloud_ex.MapStringOrder()
    trader.GetOutstandingOrders(outstanding_orders)
    u.debug("You have {} outstanding orders.".format(len(outstanding_orders)))
    # Transform outstanding orders into a DataFrame
    outstanding_orders = OutstandingOrderDF(outstanding_orders)
    return outstanding_orders
    
def historical_orders() : 
    my_historical_orders = cloud_ex.VectorOrder()
    trader.GetAllHistoricalOrders(my_historical_orders)
    u.debug("You have submitted a total of {} order(s).".format(len(my_historical_orders))) 
    my_historical_orders_df = OrderDF(my_historical_orders)
    return my_historical_orders_df

def historical_trades() : 
    my_historical_trades = cloud_ex.VectorTrade()
    trader.GetAllHistoricalTrades(my_historical_trades)
    u.debug("You have made a total of {} trade(s).".format(len(my_historical_trades)))
    my_historical_trades_df = TradeDF(my_historical_trades)
    return my_historical_trades_df

In [ ]:
outstanding_orders()

# 1. Backtesting

Let's see how we can backtest our trading algorithms to get them ready for live trading. In the following cells we will download historical data and evaluate how well a mean reversion trader would have done.

## 1.1 Get our bank of strategies

### Note on strategy and strategy parameters terminology

A strategy is implemented with specific parameters. <b>The pair of the strategy and parameters will be called an "algo"</b>
The backtest logic will take a dictionary of string keys (identifiers) to a tuple of (strategy , params). This dictionary will be called "algo_bank" 

For example => 

In [14]:
# NOte a fully specified strategy is called an "algo" and consists of a tuple of (strategy, kwargs) where kwargs provide
# the parameters for the strategy 

def mean_reversion_algo(ma,t) : 
    # Helper function for making the "algo" data structure
    # NOTE this returns a TUPLE of (strategy , kwargs)
    return (strategies_shay.strategies['mean_reversion'].strategy, { 'ma' : ma, 'threshold' : t} ) 

def momentum_algo(p1,p2,t) : 
    # NOTE this returns a TUPLE of (strategy , kwargs)
    return (strategies_shay.strategies['momentum'].strategy, { 'p1' : p1, 'p2' : p2, 'threshold' : t} ) 

def random_buy_algo(p) : 
    return (strategies_shay.strategies['random_buy'].strategy , { 'p' : p} ) 

def random_sell_algo(p) : 
    return (strategies_shay.strategies['random_sell'].strategy , { 'p' : p} ) 


# Define an "Algo Bank", which holds the (strategy, kwargs) pair, indexed by a unique identifier 
algo_bank = { 
    'mr_10_3' : mean_reversion_algo(10,3),
    'mr_10_5' : mean_reversion_algo(10,5), 
    'mr_10_10' : mean_reversion_algo(10,10),
    'mo_5_5_1' : momentum_algo(0.5,0.5,1) , 
    'mo_5_5_3' : momentum_algo(0.5,0.5,3) , 
    'mo_8_2_1' : momentum_algo(0.8,0.2,1) , 
    'mo_8_2_3' : momentum_algo(0.8,0.2,3) , 
    'rb_10'    : random_buy_algo(0.1) , 
    'rb_50'    : random_buy_algo(0.5) , 
    'rs_10'    : random_sell_algo(0.1) , 
    'rs_50'    : random_sell_algo(0.5) , 
 
}


In [15]:
def backtest_strategy_with_symbol(data,strategy, symbol, kwargs = {} ) :  
    """
    Backtests a given strategy, kwarg pair on a diven symbol using the last seconds_in_past seconds of historical data 
    """
   
    
    #bin_interval_ms = 500
    #summarize_historical_trades_df(symbol_historical_trades_df, bin_interval_ms)
    
    # Initial capital and shares we will use for the backtest. 
    init_capital = 100000
    init_shares = 1000

    # Set up trading algorithm parameters.
    num_shares = 10

    # Set up the strategy 
    trader = None
    algo = strategy(trader, [symbol],GLOBALS['BIN_INTERVAL_MS'])  # instantiates the strategy which is subclass of algorithmic_trader

    # Run the backtest.
    roi, action_list = algo.backtest(data,
                 num_shares, 
                 init_capital, 
                 init_shares, 
                 **kwargs)

    u.debug("Algo ROI={}%".format(roi))
    return roi,action_list
    
    

In [16]:
def backtest_algobank_on_symbols(algobank, symbols) : 
    """
    Backtests all algos ( strategy, kwarg pairs) on each symbol in symbols, over the last seconds_in_past seconds 
    of data 
    
    Returns the results in a sorted list with algos ranked by roi 
    """
    results = [] 
    
    end_time_ms = int(time.time()*1e3)
    start_time_ms = end_time_ms - int(GLOBALS['BACKTEST_LOOKBACK_PERIOD_SECONDS']*1e3)
    symbol_trades_vec = cloud_ex.VectorTrade()  
    cloud_ex.MarketDataAPI.PullTrades(config['project_id'], config['bigtable_id'], 
                                          config['table_name'], symbol, start_time_ms, 
                                          end_time_ms, symbol_trades_vec)
    u.debug("There are a total of {} trades for {} symbol".format(len(symbol_trades_vec), symbol))
    symbol_historical_trades_df = TradeDF(symbol_trades_vec)
    symbol_historical_trades_df = symbol_historical_trades_df.sort_values(by="CreationTimestamp")
    
    
    for symbol in symbols : 
        u.debug("\n\nBacktesting symbol: {}".format(symbol) ) 
        for (algoname,algo) in algobank.items() : 
            strategy, kwargs = algo
            u.debug("algo={}".format(algoname))
            roi,action_list = backtest_strategy_with_symbol(symbol_historical_trades_df,strategy, symbol, kwargs)
            results.append( [  roi, symbol, algoname,action_list ])
            u.debug("Made {} trades with roi: {}".format(len(action_list), roi))
    
    results.sort(key=lambda x : x[0], reverse=True) 
    return results         

            

In [ ]:
backtest_results = backtest_algobank_on_symbols(algo_bank, default_top_symbols[0:5])

In [ ]:
backtest_results

# Deploying the trading threads 

Worked more on this and completed it -- see <b>trade_algorithmically</b> function below. Nothing is tested yet so will expect bugs. But for now each thread will log its own submitted order ids to a logfile named by algo+timestamp. 

In [17]:
test_ranking = [[11, 'CQ', 'mo_8_2_3', []],
     [10,'CQ', 'mo_8_2_1', [] ] , 
     [8, 'CQ', 'mo_5_5_1', [] ] , 
     [6,'CQ', 'mr_10_10', []], 
     [6, 'CQ', 'mr_10_5', []],
     [4, 'CQ', 'mr_10_3', []]] 


In [32]:
def run_and_evaluate_algorithm(**kwargs) : 
    """
    Intended as target of new thread() 
    1. Launches the algo on the symbol and starts trading 
    2. should calculate ROI of the algo 
    4. Writes ROI and submitted order ids to disk 
    """
    name     = kwargs['name']
    strategy = kwargs['strategy']  
    strategy_parameters = kwargs['strategy_parameters'] 
    num_shares = kwargs['num_shares'] 
    max_num_orders = kwargs['max_num_orders'] 
    symbol   = kwargs['symbol']
    trader   = kwargs['trader'] #reference to the 1 trader instance connnected to cloudX
    
    # create the AlgorithmicTrader Object 
    algo = strategy(trader, [symbol], bin_interval_ms=GLOBALS['BIN_INTERVAL_MS']) 
    
    # get and set id for this trader  (for logging purposes)
    trader_id = name + "_" + str(time.time()).split(".")[0]
    #print(trader_id) 
    #print(type(algo))
    #algo.super().set_id(trader_id)  #added this method 
    
    # Calculate portfolio state pre-trading 
    pass 

    # start and finish trading
    order_ids = algo.trade(symbol,num_shares,max_num_orders,GLOBALS['WAIT_INTERVAL_MS'] ,**strategy_parameters)
    # can SIMULATE if we want 
    
    # calculate portfolio state post-trading
    pass 

    #time.sleep(10)
    # write the order ids to a log file 
    u.logfile(trader_id + "_order_ids", json.dumps(order_ids) )
    # write the ROI information a log file 
    pass     

def run_algorithm_in_thread(kwargs) : 
    # create the thread 
    t = threading.Thread(target=run_and_evaluate_algorithm,
                         kwargs=kwargs)
    # start the thread 
    t.start() 
    # return it 
    return t 

def deploy_top_N_algorithms(trader,ranked_algos, N,  num_shares, max_num_orders) : 
    sublist = ranked_algos[0:N] 
    ts = []
    for to_deploy in sublist : 
        roi, symbol, algoname , _  =  to_deploy 
        strategy, strategy_parameters = algo_bank[algoname]  
        
        arguments = { 
            'name' : algoname, 
            'strategy' : strategy, 
            'strategy_parameters' : strategy_parameters, 
            'num_shares' : num_shares, 
            'max_num_orders' : max_num_orders,  
            'symbol' : symbol , 
            'trader' : trader ,
        }
        ts.append(run_algorithm_in_thread(arguments))
        
    return ts 


def trade_algorithmically(trader,algobank,symbols,N=5) : 
    while True : 
        ranked = backtest_algobank_on_symbols(algobank,symbols)
        algo_threads = deploy_top_N_algorithms(trader,ranked,N, GLOBALS['NUM_SHARES'] , GLOBALS['MAX_NUM_ORDERS']) 
        for t in algo_threads : 
            t.join() 

In [31]:
ts = deploy_top_N_algorithms(None, test_ranking, 5, GLOBALS['NUM_SHARES'], GLOBALS['MAX_NUM_ORDERS'])
for t in ts : 
    t.join()

Running in offline mode. Could not set active symbols.
Running in offline mode. Could not set active symbols.
Running in offline mode. Could not set active symbols.
Running in offline mode. Could not set active symbols.
Running in offline mode. Could not set active symbols.


In [22]:
import importlib 
importlib.reload(sys.modules['algorithmic_trader_shay'])

<module 'algorithmic_trader_shay' from '/root/cs349f/implementation1/algorithmic_trader_shay.py'>